<a href="https://colab.research.google.com/github/Shreyas2409/Pneumonia-Prediction-using-Medical-Image-Modality/blob/master/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow
!pip install keras



In [0]:
!pip install tensorflow==2.0.0-alpha0 

In [0]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import optimizers
import tensorflow as tf 
import numpy as np
import os




In [0]:

class VGG16Extended :

  def __init__(self):
    self.base_model = VGG16(weights = 'imagenet', include_top = False)
  
  def custom_global_average_pooling2D(self, extended_model):
    return GlobalAveragePooling2D()(extended_model)
  
  def add_dense_layer_nn(self, extended_model, **kwargs):
    return Dense(units = kwargs["units"], activation =  kwargs["activation"])(extended_model)
  
  def add_dropout_layer_nn(self, extended_model, units = 0.5):
    return Dropout(units)(extended_model)
  
  def __extend_vgg16_base(self):
     extended_model = self.base_model.output #grab the outputs of VGG16 
     extended_model = self.custom_global_average_pooling2D(extended_model)
     extended_model = self.add_dense_layer_nn(extended_model, units = 512, activation = "relu")
     extended_model = self.add_dropout_layer_nn(extended_model)
     extended_model = self.add_dense_layer_nn(extended_model, units = 64, activation = "relu")
     extended_model = self.add_dense_layer_nn(extended_model, units = 2, activation = 'sigmoid')

     return Model(inputs = self.base_model.input, outputs = extended_model)
    
  def compile(self):
    model = self.__extend_vgg16_base()
    model.compile(
        loss = "categorical_crossentropy",
        optimizer = optimizers.SGD(lr = 1e-4, momentum = 0.9),
        metrics = ['accuracy']
    )


In [0]:


img_width, img_height = 250, 250
train_data = 'home/shreyas/Desktop/Pneumonia-Predirediction-using-Medical-Image-Modality/chest_xray/train'
test_data = 'home/shreyas/Desktop/Pneumonia-Predirediction-using-Medical-Image-Modality/chest_xray/test'
val_data = 'home/shreyas/Desktop/Pneumonia-Predirediction-using-Medical-Image-Modality/chest_xray/val'

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')


# GPU scheduling 
with tf.device("/device:GPU:0"):
   # on GPU 0
   model = VGG16Extended().compile()
   history_buffer = model.fit_generator(
       train_generator,
       epochs = 20, shuffle = True, verbose = 1, validation_data = test_generator
   )


# New Section